In [ ]:
import os, sys
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta

In [ ]:
file_path = r"C:/Users/Sarthak.Sachdev/Desktop/python-code/notebook/datadump/AmeyoReports/acd_call_details_nov_feb.csv"

dfCallHistory = pd.read_csv(file_path, low_memory=False)

print("File loaded successfully!")

In [ ]:
dfCallHistory.head(10)

In [ ]:
dfCallHistory['Call Time'] = pd.to_datetime(dfCallHistory['Call Time'], dayfirst=True, errors='coerce')

In [ ]:
dfCallHistory['Call Date'] = pd.to_datetime(dfCallHistory['Call Time']).dt.date

In [ ]:
dfCallHistory['date'] = dfCallHistory['Call Time'].dt.strftime('%d')

In [ ]:
dfCallHistory['Call Hour'] = pd.to_datetime(dfCallHistory['Call Time']).dt.hour

In [ ]:
dfCallHistory['Call Month'] = pd.to_datetime(dfCallHistory['Call Time']).dt.month.astype(str)

In [ ]:
dfCallHistory

In [ ]:
dfCallHistory['date'] = dfCallHistory['date'].astype(int)

In [ ]:
# Define Week Number

conditions = [
    (dfCallHistory['date'] >=1) & (dfCallHistory['date'] <=7),
    (dfCallHistory['date'] >=8) & (dfCallHistory['date'] <=14),
    (dfCallHistory['date'] >=15) & (dfCallHistory['date'] <=21),
    (dfCallHistory['date'] >=22) & (dfCallHistory['date'] <=28),
    (dfCallHistory['date'] >=29) 
]

choices = ['W1', 'W2', 'W3', 'W4', 'W5']

dfCallHistory['Week'] = np.select(conditions, choices, default = 'NA')

In [ ]:
dfCallHistory

In [ ]:
dfCallHistory['Wait Time at ACD (sec)'] = pd.to_timedelta(dfCallHistory['Wait Time at ACD']).dt.total_seconds()
dfCallHistory['Total Wait Time (sec)'] = pd.to_timedelta(dfCallHistory['Total Wait Time']).dt.total_seconds()
dfCallHistory['Customer Hold Duration (sec)'] = pd.to_timedelta(dfCallHistory['Customer Hold Duration']).dt.total_seconds()
dfCallHistory['User Setup Time (sec)'] = pd.to_timedelta(dfCallHistory['User Setup Time']).dt.total_seconds()
dfCallHistory['User Ringing Time (sec)'] = pd.to_timedelta(dfCallHistory['User Ringing Time']).dt.total_seconds()
dfCallHistory['User Hold Duration (sec)'] = pd.to_timedelta(dfCallHistory['User Hold Duration']).dt.total_seconds()
dfCallHistory['User Talk Time (sec)'] = pd.to_timedelta(dfCallHistory['User Talk Time']).dt.total_seconds()
dfCallHistory['ACW Duration (sec)'] = pd.to_timedelta(dfCallHistory['ACW Duration']).dt.total_seconds()

In [ ]:
dfCallHistory

In [ ]:
dfCallHistory['AHT_sec'] = dfCallHistory['User Talk Time (sec)'] + dfCallHistory['ACW Duration (sec)'] + dfCallHistory['Customer Hold Duration (sec)'] + dfCallHistory['User Hold Duration (sec)']

In [ ]:
dfCallHistory

In [ ]:
columns_to_drop = ['#', 'Campaign Name', 'Other Filter Groups', 'Applied filter group', 'Table Filters', 'DNIS', 'Call Type', 'Call ID', 'Queue Name', 'Hangup Details', 'Actual Channel', 'Username', 'Queue ID', 'Call Notes', 'Call Time', 'User Disposition Code', 'date', 'Call Month', 'Wait Time at ACD', 'Total Wait Time', 'Customer Hold Duration', 'User Hold Duration', 'User Talk Time', 'ACW Duration', 'User Setup Time', 'User Ringing Time', 'User Hold Duration (sec)', 'Customer Hold Duration (sec)', 'ACW Duration (sec)', 'User Talk Time (sec)', 'Phone']

dfCallHistory['Call Month'] = pd.to_numeric(dfCallHistory['Call Month'], errors='coerce')

dfCallHistory_filtered = dfCallHistory[dfCallHistory['Call Month'] == 1]

dfCallHistory_filtered = dfCallHistory_filtered.drop(columns=columns_to_drop)

dfCallHistory_filtered

In [ ]:
dfCallHistory_filtered['answered_flag'] = dfCallHistory_filtered['Answered/Hungup'].apply(lambda x: 1 if x == 'ANSWERED' else 0)

dfCallHistory_filtered.drop(columns = ['Answered/Hungup'], inplace=True)

In [ ]:
dfCallHistory_filtered['Cumulative User Talk Time'] = dfCallHistory_filtered['Cumulative User Talk Time'].astype(str)

def convert_to_seconds(time_str):
    try:
        time_parts = time_str.split(':')
        if len(time_parts) == 3:
            return sum(int(i) * 60 ** (2 - idx) for idx, i in enumerate(time_parts))
        else:
            raise ValueError(f"Invalid time format: {time_str}. Expected format: hh:mm:ss")
    except Exception as e:
        raise ValueError(f"Error processing time value '{time_str}': {e}")

# Apply the conversion function
dfCallHistory_filtered['Cumulative_User_Talk_Time_sec'] = dfCallHistory_filtered['Cumulative User Talk Time'].apply(convert_to_seconds)

dfGrouped_sorted = dfCallHistory_filtered.drop(columns=['Cumulative User Talk Time'])

In [ ]:
dfGrouped_sorted = dfCallHistory_filtered.groupby(['User ID', 'Call Date', 'Call Hour']).agg(
    answered_calls=('answered_flag', 'sum'),
    Cumulative_User_Talk_Time=('Cumulative_User_Talk_Time_sec', 'sum'),
    Wait_Time_at_ACD_sec=('Wait Time at ACD (sec)', 'sum'),
    Total_Wait_Time_sec=('Total Wait Time (sec)', 'sum'),
    User_Setup_Time_sec=('User Setup Time (sec)', 'sum'),
    User_Ringing_Time_sec=('User Ringing Time (sec)', 'sum'),
    AHT_sec=('AHT_sec', 'sum'),
    total_calls=('User ID', 'size')
).reset_index()

dfGrouped_sorted = dfGrouped_sorted.sort_values(by=['User ID', 'Call Date', 'Call Hour'])

dfGrouped_sorted

In [ ]:
write("https://docs.google.com/spreadsheets/d/1pSXAulxdvIxMX_7drSRi_e2Ya-sAod-oYDgbt4IGrAY/edit?gid=975529161#gid=975529161", "DSC agentwise data", dfGrouped_sorted)

In [ ]:
(# Dod Calls

(dfCallHistory[(dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"])) & (dfCallHistory['Answered/Hungup']=="ANSWERED") & (dfCallHistory['User Talk Time (sec)'] > 30)].groupby(['Call Date']).agg(totalCalls = ('Call ID','count'), AHT = ('User Talk Time (sec)', 'mean') ).reset_index())).to_clipboard()


In [ ]:
# Total Calls

dfCallHistory[(dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"]))].groupby(['Call Month']).agg(totalCalls = ('Call ID','count'))

In [ ]:
# Overall Answered, AWT - Missed , AWT - Answered, AHT - Overall 

dfCallHistory[(dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"])) & (dfCallHistory['Answered/Hungup']=="ANSWERED")].groupby(['Call Month', 'User ID', 'Answered/Hungup']).agg(call_count = ('Call ID', 'count'), AHT = ('AHT_sec', 'mean'), AWT = ('Wait Time at ACD (sec)', 'mean')).reset_index()

In [ ]:
# Answered Calls > 30 sec - Count and AHT

dfCallHistory[(dfCallHistory['User Talk Time (sec)'] > 30) & (dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"])) ].groupby(['Call Month', 'Answered/Hungup']).agg(call_count = ('Call ID', 'count'), AHT_sec = ('AHT_sec', 'mean')).reset_index()

In [ ]:
# Agent Ring Duration

dfCallHistory[(dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"]))].groupby(['Call Month']).agg(ring_duration = ('User Ringing Time (sec)', 'mean')).reset_index()

In [ ]:
# Missed by Agent - CUSTOMER_HANGUP_PHONE + AGENT_HANGUP_UI + AGENT_HANGUP_PHONE

((dfCallHistory[(dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"])) & (dfCallHistory['Answered/Hungup'] == "HUNGUP") & (dfCallHistory['Hangup Details'].isin(["AGENT_HANGUP_PHONE", "AGENT_HANGUP_UI", "CUSTOMER_HANGUP_PHONE"]))].groupby(['Call Month', 'Answered/Hungup']).agg(call_count = ('Call ID', 'count'), AWT = ('Wait Time at ACD (sec)', 'mean')).reset_index()))

In [ ]:
dfCallHistory[(dfCallHistory['User Talk Time (sec)'] > 30) & (dfCallHistory['Queue Name'].isin(["Driver_Support_Queue", "Driver_Queue"]))].groupby(['Call Month','Queue Name', 'Answered/Hungup']).agg(call_count = ('Call ID', 'count'), AHT = ('AHT_sec', 'mean')).reset_index()

# Outbound Calls

In [ ]:
dfOutbound = pd.read_csv(r'../datadump/AmeyoReports/OBOct.csv', low_memory=False)

In [ ]:
dfOutbound

In [ ]:
dfOutbound['Call Type'].unique()

In [ ]:
temp = ['outbound.callback.dial','outbound.manual.dial', 'outbound.manual.preview.dial']

In [ ]:
dfOutbound = dfOutbound[dfOutbound['Call Type'].isin(temp)]

In [ ]:
dfOutbound

In [ ]:
dfOutbound['Call Time'] = pd.to_datetime(dfOutbound['Call Time'],  dayfirst = True, errors = 'coerce')

In [ ]:
dfOutbound

In [ ]:
dfOutbound['Call Date'] = pd.to_datetime(dfOutbound['Call Time'],  dayfirst=True)

In [ ]:
dfOutbound

In [ ]:
dfOutbound['Call Date'] = dfOutbound['Call Date'].dt.date

In [ ]:
dfOutbound

In [ ]:
dfOutbound['Month'] = dfOutbound['Call Time'].dt.strftime('%Y-%m')

In [ ]:
((dfOutbound.groupby(['Month', 'Call Type']).agg( aa = ('Call ID','nunique'), count = ('Call ID','count') ).reset_index()))

In [ ]:
dfOutbound['date'] = dfOutbound['Call Time'].dt.strftime('%d')
dfOutbound['date'] = dfOutbound['date'].astype(int)

In [ ]:
# Define Week Number

conditions = [
    (dfOutbound['date'] >=1) & (dfOutbound['date'] <=7),
    (dfOutbound['date'] >=8) & (dfOutbound['date'] <=14),
    (dfOutbound['date'] >=15) & (dfOutbound['date'] <=21),
    (dfOutbound['date'] >=22) & (dfOutbound['date'] <=28),
    (dfOutbound['date'] >=29) 
]

choices = ['W1', 'W2', 'W3', 'W4', 'W5']

dfOutbound['Week'] = np.select(conditions, choices, default = 'NA')

In [ ]:
dfOutbound['Customer Setup Time (sec)'] = pd.to_timedelta(dfOutbound['Customer Setup Time']).dt.total_seconds()
dfOutbound['Customer Ringing Time (sec)'] = pd.to_timedelta(dfOutbound['Customer Ringing Time']).dt.total_seconds()
dfOutbound['IVR Time (sec)'] = pd.to_timedelta(dfOutbound['IVR Time']).dt.total_seconds()
dfOutbound['Customer Talk Time (sec)'] = pd.to_timedelta(dfOutbound['Customer Talk Time']).dt.total_seconds()
dfOutbound['Customer Hold Duration (sec)'] = pd.to_timedelta(dfOutbound['Customer Hold Duration']).dt.total_seconds()
dfOutbound['User Setup Time (sec)'] = pd.to_timedelta(dfOutbound['User Setup Time']).dt.total_seconds()
dfOutbound['User Ringing Time (sec)'] = pd.to_timedelta(dfOutbound['User Ringing Time']).dt.total_seconds()
dfOutbound['User Talk Time (sec)'] = pd.to_timedelta(dfOutbound['User Talk Time']).dt.total_seconds()
dfOutbound['ACW Duration (sec)'] = pd.to_timedelta(dfOutbound['ACW Duration']).dt.total_seconds()

In [ ]:
dfOutbound

In [ ]:
dfOutbound.groupby(['Month', 'Campaign Name'] ).agg(call_count = ('Call ID', 'count'), unique_calls = ('Call ID', 'nunique')).reset_index()

In [ ]:
tempDf = dfOutbound.groupby(['Call ID']).agg(call_count = ('Call Time','count')).reset_index()

In [ ]:
tempDf[tempDf['call_count']!=1]

In [ ]:
dfOutbound[dfOutbound['Call ID']=="d390-66d7613a-vcall-1009482"]

In [ ]:
dfOutbound['User ID'].unique()

In [ ]:
dfPSC = read("https://docs.google.com/spreadsheets/d/1SD4IXn6v5t4wADY5d4_Sm10IeiKt087__mgNirI6Zww/edit?gid=0#gid=0", "Sheet1")

In [ ]:
dfPSC

In [ ]:
reqCols = ['email', 'PSC Flag']

dfPSC = dfPSC[reqCols]

In [ ]:
dfPSC

In [ ]:
dfOutbound = dfOutbound.merge(dfPSC, left_on = 'User ID', right_on = 'email', how = 'left')

In [ ]:
dfOutbound['PSC Flag'] = dfOutbound['PSC Flag'].fillna(0)

In [ ]:
dfOutbound.groupby(['Month', 'PSC Flag']).agg(unique_calls = ('Call ID','nunique'), call_count = ('Call ID','count')).reset_index()

In [ ]:
dfDSCOutbound = dfOutbound[dfOutbound['PSC Flag']==0]

In [ ]:
dfDSCOutbound.info()

In [ ]:
dfDSCOutbound['AHT_sec'] = dfDSCOutbound['Customer Setup Time (sec)'] + dfDSCOutbound['Customer Ringing Time (sec)'] + dfDSCOutbound['User Setup Time (sec)'] + dfDSCOutbound['User Ringing Time (sec)'] + dfDSCOutbound['User Talk Time (sec)'] + dfDSCOutbound['ACW Duration (sec)'] + dfDSCOutbound['Customer Hold Duration (sec)']

In [ ]:
dfDSCOutboundGr = dfDSCOutbound.groupby(['Month','Call ID', 'Call Type']).agg(instances = ('Call ID', 'count'), 
                                                        customer_setup_time_sec = ('Customer Setup Time (sec)', 'mean'),
                                                        customer_ringing_time_sec = ('Customer Ringing Time (sec)', 'mean'),
                                                        customer_talk_time_sec = ('Customer Talk Time (sec)', 'mean'),
                                                        customer_hold_duration_sec = ('Customer Hold Duration (sec)', 'mean'),
                                                        user_setup_time_sec = ('User Setup Time (sec)', 'sum'),
                                                        user_ringing_time_sec = ('User Ringing Time (sec)', 'sum') ,
                                                        user_talk_time_sec = ('User Talk Time (sec)', 'sum'),
                                                        ACW_duration_sec = ('ACW Duration (sec)', 'sum')
                                                        ).reset_index()

In [ ]:
dfDSCOutboundGr

In [ ]:
dfDSCOutboundGr['AHT_sec'] = dfDSCOutboundGr['customer_setup_time_sec'] + dfDSCOutboundGr['customer_ringing_time_sec'] + dfDSCOutboundGr['user_setup_time_sec'] + dfDSCOutboundGr['user_ringing_time_sec'] + dfDSCOutboundGr['user_talk_time_sec'] + dfDSCOutboundGr['ACW_duration_sec'] + dfDSCOutboundGr['customer_hold_duration_sec']

In [ ]:
dfDSCOutboundGr[dfDSCOutboundGr['instances']!=1]

In [ ]:
dfDSCOutboundGr[(dfDSCOutboundGr['Call Type']=='outbound.manual.dial') & (dfDSCOutboundGr['instances']!=1)]

In [ ]:
dfDSCOutboundGr['Call Type'].unique()

In [ ]:
dfDSCOutboundGr.groupby('Month').agg({'Call ID':'nunique'}).reset_index()

In [ ]:
dfDSCOutboundGr.groupby('Month').agg({'instances':'sum'}).reset_index()

In [ ]:
dfDSCOutboundGr.groupby('Month').agg(connected_call_count = ('instances', 'sum'), handling_time = ('AHT_sec', 'sum'), Avg_handling_time = ('AHT_sec', 'mean')).reset_index()

In [ ]:
dfDSCOutbound[dfDSCOutbound['System Disposition']=='CONNECTED'].groupby('Month').agg(connected_call_count = ('Call ID', 'count')).reset_index()

In [ ]:
dfDSCOutbound.groupby(['Month','System Disposition']).agg(call_count = ('Call ID', 'count')).reset_index()

In [ ]:
dfDSCOutbound[(dfDSCOutbound['System Disposition']=='CONNECTED')].groupby('Month').agg(connected_call_count = ('Call ID', 'count'), aht = ('AHT_sec', 'mean')).reset_index()

In [ ]:
dfDSCOutbound[(dfDSCOutbound['System Disposition']=='CONNECTED') & (dfDSCOutbound['User Talk Time (sec)']<= 30) ].groupby('Month').agg(connected_call_count = ('Call ID', 'count')).reset_index()